In [4]:
import os
import re
import json

import requests
import streamlit as st
from groq import Groq
from twelvelabs import TwelveLabs

In [ ]:
class TrailMetadata:
    def __init__(self):
        self.trails = json.load(open("cotrex.json"))

    def get_trail_by_timestamp(self, seconds_timestamp):
        minutes_timestamp = seconds_timestamp / 60
        for trail in self.trails:
            if trail["segment-start"] <= minutes_timestamp < trail["segment-end"]:
                trail.pop("segment-start")
                trail.pop("segment-end")
                trail.pop("map-thumbnail")
                print("trail: ",trail)
                return trail
        return None

In [11]:
class SearchResult:
    def __init__(self, item, youtube_link):
        self.thumbnail_url = item.thumbnail_url
        self.local_thumbnail_path = get_local_thumbnail(self.thumbnail_url)
        self.youtube_link = f"{youtube_link}&t={item.start}s"
        self.metadata = item.metadata
        self.start_time = item.start
        self.youtube_embed = f"""
        <embed width=“320” height=“240”
        source src=“{self.youtube_link}” title=“YouTube video player” frameborder=“0” allow=“accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture” allowfullscreen>
        """

        trail_metadata = TrailMetadata()
        trail_info = trail_metadata.get_trail_by_timestamp(self.start_time)
        self.trail_info = trail_info

    def __repr__(self):
        str_repr = ""
        str_repr += f"thumbnail_url={self.thumbnail_url}\n"
        str_repr += f"youtube_link={self.youtube_link}\n"
        str_repr += f"metadata={self.metadata}\n"
        str_repr += f"local_thumbnail_path={self.local_thumbnail_path}\n"
        str_repr += f"youtube_embed={self.youtube_embed}\n"
        str_repr += f"start_time={self.start_time}\n"
        str_repr += f"trail_info={self.trail_info}\n"
        return str_repr

    def for_model(self):
        return f"""{self.metadata}
{self.trail_info}
"""

In [3]:
TL_INDEX_ID = os.getenv("TL_INDEX_ID")
print(TL_INDEX_ID)

66ae787b6222fe7b53fd7880


In [ ]:
def search(
        query,
        index_id=TL_INDEX_ID,
        options=["visual", "conversation", "text_in_video"],
        youtube_link="https://www.youtube.com/watch?v=KKeZPA-Gvs4",
        n=1,
) -> SearchResult:
    client = TwelveLabs(api_key=TL_INDEX_ID)
    print("client:", client)
    result = client.search.query(
        index_id=index_id,
        query=query,
        options=options,
        page_limit=n,
    )
    print("result:", result)

    top_hit = result.data[0]
    search_result = SearchResult(top_hit, youtube_link)

    return search_result
